In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Basic overview of dataset

Submission outline:

In [ ]:
sample_submission = pd.read_csv("../input/nbme-score-clinical-patient-notes/sample_submission.csv")
sample_submission

## Feature dataset

In [ ]:
features = pd.read_csv("../input/nbme-score-clinical-patient-notes/features.csv")
features.head()

In [ ]:
features["case_num"].unique()

## explaination of each feature:
1. feature_num, a unique identifier
2. case_num, a unique number for each case, 即分类类别
3. feature_text, 命名体识别对应的text

用一个简单的例子查看一下:


In [ ]:
features.iloc[0,:]["feature_text"]

## patient notes explaination
A collection of about 40,000 Patient Note history portions. Only a subset of these have features annotated. You may wish to apply unsupervised learning techniques on the notes without annotations. The patient notes in the test set are not included in the public version of this file.

1. pn_num: id， 40000个patient note的对应id
2. case_num: 不同的case
3. pn_history: 即对应的文本内容，里面的\r\n 即为换行符(windows中特定)

In [ ]:
patient_notes = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")
patient_notes.head().iloc[0]["pn_history"]

In [ ]:
patient_notes

In [ ]:
#最长的就194个词？？？
patient_notes["pn_history"].apply(lambda x: len(x.split())).max()

## CHECK train.csv

可以看到，14300条数据中有9901条是有结果的

In [ ]:
train = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
train_1 = train.copy()
train

In [ ]:
train[train["annotation"] != "[]"]

In [ ]:
test = pd.read_csv("../input/nbme-score-clinical-patient-notes/test.csv")
test = pd.merge(test, features, on = ["feature_num", "case_num"], how = "left")
test = pd.merge(test, patient_notes, on = ["pn_num", "case_num"], how = "left") 
test

## Merge the dataset together

In [ ]:
train = pd.merge(train,features, on = ["feature_num", "case_num"], how = "left")
train = pd.merge(train, patient_notes, on = ["pn_num", "case_num"], how = "left") 

In [ ]:
train

In [ ]:
import ast
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)

## Do some checking for each word location and feature text, check whether each of them is correctly labling

Here I save \r, \n since when being a new line, it maybe helpful for our model to detect those key info

In [ ]:
def correct_true_label(annotation, location, raw_text):
    '''
    Input is:
        1. annotation, the corresponding annotation
        2. location, each location to annotation, assume to be a string "a b"
        3. raw_text, corresponding raw text
    '''
    # 将标注错，标注漏的数据correct
    ls = [",",".",";"," ","\r","\n","\r\n"]
    idx_start = int(location.split(" ")[0])
    idx_end = int(location.split(" ")[1])
    true_text = raw_text[idx_start:idx_end]
    
    
    # ------------------------------ 这个while loop 判断index start的问题
    # 对于这个 while loop，判断start 是不是在ls 里面，如果是，则说明其起始并不是字母或者数字
    # 需要修正为字母，数字
    while(1):
        if raw_text[idx_start] in ls:
            idx_start += 1
        # 如果start 不为0 而且 start -1 不在ls里面，说明前面有漏标的数据，需要修正 比如 father为原文， 标注成了 ather
        elif idx_start >0 and raw_text[idx_start - 1] not in ls:
            idx_start -= 1
        else:
            break
            
            
    # --------------------------------------这个while loop 是判断idx end的问题
    # 首先如果idx end 不为原始数据 raw text 的结尾
    # 然后判断结尾是否在ls 里面，如果在，说明标注问题，需要往前移一位
    # 然后判断结尾 +1 是否在ls里面，如果不在，说明标注漏了， 需要往后移一位
    while(1):
        #print(idx_end)
        #print(raw_text[idx_end])
        if idx_end < len(raw_text):
            if raw_text[idx_end] in ls:
    #             print("in if")
    #             print(raw_text[idx_end])
                idx_end -=1
            elif idx_end +1 < len(raw_text) and raw_text[idx_end + 1] not in ls:
    #             print("in elif")
    #             print(raw_text[idx_end + 1])
                idx_end +=1
            else:
                break
        else:
            if raw_text[idx_end - 1] not in ls:
                idx_end -=1
    annotation = raw_text[idx_start:idx_end+1]
    #print(annotation, idx_start, idx_end+1)
    return annotation, idx_start, idx_end+1

In [ ]:
def checking(annotation, location, raw_text):
    assert len(annotation) == len(location), "sth wrong, check the length!"
    correct_format = []
    correct_loc = []
#     print(annotation)

#----------------- 这里是返回更新后的最新的 location 和 annotation
    for anno, idx in zip(annotation, location):
        #print(idx)
        if ";" in idx:
            for j in idx.split(";"):
                count = 0
                corr_anno, start,end = correct_true_label(anno, j, raw_text)
                #print(count)
                #print(corr_anno, start, end)
                correct_format.append(corr_anno[count:count+int(end) - int(start)])
                correct_loc.append([start, end])
                count += int(start) - int(end)+1
            # count the correct index for annotation
        else:
            corr_anno, start,end = correct_true_label(anno, idx, raw_text)
            correct_loc.append([start, end])
            correct_format.append(corr_anno)
    assert len(correct_format) == len(correct_loc)
    # check the len
#     print(correct_format)
#     print(correct_loc)

    # ------------------------ 这里是检查选取出来的长度和实际的长度是否相同
     # ----------------- 检查选取出来的结果和 实际是否对应
    tmp =True
    for anno, loc in zip(correct_format,correct_loc):
#         print(anno, loc)
#         print(len(anno), int(loc[1]) - int(loc[0]))
        len_ = int(loc[1]) - int(loc[0])
        if len_ != len(anno):
            tmp = False
        # 比较完长度之后，从对应的原文中选出对应的值，查看两个string是否相等
        if raw_text[int(loc[0]):int(loc[1])] != anno:
            tmp = False
    # 检查完确认没问题后，返回对应的全新的 annotation 和 location
    return {"annotation":correct_format, "location":correct_loc}

In [ ]:
def clean_feature(text):
    '''
    Clean the feature text
    '''
    text = re.sub("-OR-", " or ", text)
    text = re.sub('I-year', '1-year', text)
    text = re.sub("-", " ", text)
    return text

In [ ]:
train["feature_text"] = train.apply(lambda x: clean_feature(x["feature_text"]), axis = 1)
test["feature_text"] = test.apply(lambda x: clean_feature(x["feature_text"]), axis = 1)

In [ ]:
tmp = train.apply(lambda x: checking(x["annotation"], x["location"], x["pn_history"]), axis=1)
train["annotation"] = tmp.apply(lambda x: x["annotation"])
train["location"] = tmp.apply(lambda x: x["location"])
# check how many answer for specific question
train["annotation_length"] = train.apply(lambda x: len(x["annotation"]), axis = 1)

In [ ]:
train

## Tokenizer

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModel

In [ ]:
def get_tokenizer(name, precompute=False, df=None, path=None):
    '''
    
    '''
    if path is None:
        tokenizer = AutoTokenizer.from_pretrained(name)
    else:
        tokenizer = AutoTokenizer.from_pretrained(path)

    tokenizer.name = name

    tokenizer.special_tokens = {
        "sep": tokenizer.sep_token_id,
        "cls": tokenizer.cls_token_id,
        "pad": tokenizer.pad_token_id,
    }

    if precompute:
        tokenizer.precomputed = precompute_tokens(df, tokenizer)
    else:
        tokenizer.precomputed=None
        
    return tokenizer

def precompute_tokens(df, tokenizer):
    
    feature_text = df["feature_text"]
    idx = {}
    offsets = {}
    for txt in feature_text:
        encoding = tokenizer(
            txt,
            return_token_type_ids = True,
            return_offsets_mapping=True,
            return_attention_mask=False,
            add_special_tokens=False,
        )
        idx[txt] = encoding["input_ids"]
        # character Index
        offsets[txt] = encoding["offset_mapping"]
        
    for txt in df["pn_history"].unique():
        encoding = tokenizer(
            txt,
            return_token_type_ids = True,
            return_offsets_mapping=True,
            return_attention_mask=False,
            add_special_tokens=False,
        )
        idx[txt] = encoding["input_ids"]
        offsets[txt] = encoding["offset_mapping"]
    return {"idx":idx, "offsets":offsets}
    
def robert_process(feature_text, text, pre_computed, tokenizer, max_length = 300, location = None, anno_len = None):
    '''
    Preprocess it as a Question answering problem, where question is feature_text, answer in text file
    
    This function will change the original labels to token labels:
    For this kind of question_answering problem:
    Notice that:
        1. For each question(which is feature_text here), have multiple discrete solutions, [[123,124], [145, 146]] for example
        2. We cannot just map it as start logit and end logit, which means we will use the whole length as label: if our text length is 5, then the lables will like this:
            [0,1,1,-1,-1], where 0 means not in answer, 1 means in answer, -1 means padding
    '''
    tokens = tokenizer.special_tokens
    sep = [tokens["sep"], tokens["sep"]]
    # for input_ids
    input_ids = [tokens["cls"]] + pre_computed["idx"][feature_text] + sep
    question_len = len(input_ids)
    #print(question_len)
    # add answer text
    input_ids += pre_computed["idx"][text] 
    #print(len(input_ids))
    input_ids = input_ids[:max_length -1] + [tokens["sep"]]
    # segment id
    # since roberta don't have tokens id, so just all 0 is ok
    token_type_ids = [0] * len(input_ids)
    
    # for character index
    # don't care feature text, only for answer text
    offsets = [(0,0)] * question_len + pre_computed["offsets"][text]
    # for end 
    offsets = offsets[:max_length -1] + [(0,0)]
    
    # add the padding
    padding_length = max_length - len(input_ids)
    
    if padding_length > 0 :
        input_ids +=  [tokens["pad"]] * padding_length
        token_type_ids += [0] * padding_length
        offsets += [(0,0)] * padding_length
    #print(offsets)
    # create labels
    if location is not None:
        labels = np.zeros_like(input_ids)
        # set all question to 0 , special token, padding to -1
        labels[0] = -1
        labels[1:question_len] = 0
        labels[question_len] = -1
        labels[question_len-1] = -1
        labels[max_length - padding_length-1:] = -1
        if anno_len != 0:
            for loc in location:
                    start_idx = -1
                    end_idx = -1
                    start, end = int(loc[0]), int(loc[1])
                    #print(start, end)
                    for idx in range(len(offsets)):
                        #choose the index 
                        if (start_idx == -1) & (start < offsets[idx][0]):
                            start_idx = idx - 1
                        if (end_idx == -1) & (end <= offsets[idx][1]):
                            end_idx = idx + 1
                    if start_idx == -1:
                        start_idx = end_idx
                    if (start_idx != -1) & (end_idx != -1):
                        labels[start_idx:end_idx] = 1
            #print(start, end, start_idx, end_idx)
                 
        assert len(input_ids) == len(token_type_ids) == len(offsets) == max_length == len(labels)
        return {
            "input_ids": input_ids,
            "segment_ids":token_type_ids,
            "offsets": offsets,
            "labels":labels
        }
    assert len(input_ids) == len(token_type_ids) == len(offsets) == max_length
    return  {
            "input_ids": input_ids,
            "segment_ids":token_type_ids,
            "offsets": offsets
            }

In [ ]:
tokenizer = get_tokenizer("roberta-large", True, train, "../input/roberta-baseline-params")
tokenizer_test = get_tokenizer("roberta-large", True, test, "../input/roberta-baseline-params")

In [ ]:
def check_max_length(feature_text, text, pre_computed, tokenizer):
    tokens = tokenizer.special_tokens
    sep = [tokens["sep"], tokens["sep"]]
    # for input_ids
    input_ids = [tokens["cls"]] + pre_computed["idx"][feature_text] + sep
    #print(question_len)
    # add answer text
    input_ids += pre_computed["idx"][text] 
    #print(len(input_ids))
    input_ids = input_ids + [tokens["sep"]]
    return len(input_ids)
max_length = train.apply(lambda x:check_max_length(x["feature_text"], x["pn_history"], tokenizer.precomputed, tokenizer), axis=1).max()
max_length

## Create torch dataset

In [ ]:
import torch
class Patient_note(torch.utils.data.Dataset):
    def __init__(self, train_set, tokenizer, max_length):
        self.df = train_set
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        text = self.df.iloc[idx]["pn_history"]
        feature_text = self.df.iloc[idx]["feature_text"]
        anno_len = self.df.iloc[idx]["annotation_length"]
        location = self.df.iloc[idx]["location"]
        #print(location)
        encoding = robert_process(feature_text, text,self.tokenizer.precomputed, self.tokenizer, self.max_length, location, anno_len)
        return torch.tensor(encoding["input_ids"]), torch.tensor(encoding["segment_ids"]), torch.tensor(encoding["labels"]) ,np.array(encoding["offsets"]), text
train_Set = Patient_note(train, tokenizer, max_length)
_,_, labels, offset, _ = next(iter(train_Set))

In [ ]:
class Test_dataset(torch.utils.data.Dataset):
    def __init__(self, test_set, tokenizer, max_length):
        self.df = test_set
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        text = self.df.iloc[idx]["pn_history"]
        feature_text = self.df.iloc[idx]["feature_text"]
        encoding = robert_process(feature_text, text,self.tokenizer.precomputed, self.tokenizer, self.max_length)
        return torch.tensor(encoding["input_ids"]), torch.tensor(encoding["segment_ids"]), np.array(encoding["offsets"])
test_set = Test_dataset(test, tokenizer_test, max_length)
#next(iter(test_set))

In [ ]:
from torch import nn
class Roberta_baseline(nn.Module):
    def __init__(self, model_name, num_class, pretrained = True, path = None):
        super().__init__()
        # basic model from transformers
        if pretrained:
            self.model = AutoModel.from_pretrained(model_name)
        else:
            self.model = AutoModel.from_pretrained(path)
        self.ln1 = nn.Linear(1024, num_class)
        self.padding_idx = 1
    def forward(self, input_ids, segment_ids):
        # get last hidden state as output, which is (batch_size, seq_len, hidden_size)
        logit = self.model(
            input_ids,
            attention_mask = (input_ids != self.padding_idx).long(),
            token_type_ids = segment_ids)[0]
       # print(logit.shape)
        logit = self.ln1(logit)
        return logit

In [ ]:
# net = Roberta_baseline("roberta-large", 1, False,"../input/roberta-baseline-params")
# dic = torch.load("../input/roberta-model-para/model.params")
# net.load_state_dict(dic)

In [ ]:
def post_process(predictions, offset_mapping, tolerance = 0.5):
    '''
    This function is the post processing after inference from trained model:
    input params:
        1. offset_mapping: the result index
        2. tolerance, the sigmoid tolerance
    '''
    result = []
    for i in range(predictions.shape[0]):
        idx = np.argwhere(predictions[i,:,:]>tolerance)
    #print(idx[:,1])
    #print(offset_mapping.shape)
        result.append(offset_mapping[i,idx[:,0],:])
    return result
def inference(net, test_set, device):
    '''
    input params:
        1. net, trained net
        2. test_set
        3. device
    '''
    # since we only have 5 test sample
    result = []
    test_iter = torch.utils.data.DataLoader(test_set, batch_size = 5)
    net.eval()
    net.to(device)
    for idx, value in tqdm(enumerate(test_iter)):
        input_ids = value[0].to(device)
        seg_ids = value[1].to(device)
        with torch.no_grad():
            y_preds = net(input_ids, seg_ids)
        result.append(post_process(y_preds.sigmoid().to('cpu').numpy(), value[2].to("cpu").numpy()))
    return result
# predictions = inference(net, test_set, torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))

In [ ]:
def get_return(predictions):
    '''
    This function receive the prediction and will return the merge index.
    input_prams:
        predictions: a list with test data
    '''
    result = []
    for prediction in predictions:
        for i in range(len(prediction)):
            preds = prediction[i]
            ls = []
            if preds.tolist() != []:
                #print(preds, type(preds))
                start, end = preds[0,0], preds[0,1]
                for j in range(preds.shape[0]):
                    if j+1 >= len(preds):
                        break
                    else:
                        if preds[j+1,0] - preds[j, 1] == 1 or preds[j+1,0] - preds[j, 1] == 0:
                            end = preds[j+1, 1]
                            continue
                        else:
                            ls.append(str(start)+" "+str(end))
                            start = preds[j+1, 0]
                            end = preds[j+1, 1]
                ls.append(str(start)+ " " + str(end))
                if ls == [str(start)+ " " + str(end)]:
                    result.append(str(start) + " " + str(end))
                else:
                    if start - int(ls[-2].split(" ")[-1]) == 1 or start - int(ls[-2].split(" ")[-1]) == 0:
                        ls[-1] = ls[-2].split(" ")[0] + " " + str(end)
                        result.append(";".join(ls))
                    else:
                        result.append(";".join(ls))
            else:
                result.append(np.NaN)
    return result

In [ ]:
# results = pd.Series(get_return(predictions))
# results

In [ ]:
# submission = pd.read_csv("../input/nbme-score-clinical-patient-notes/sample_submission.csv")
# submission["location"] = results
# display(submission.head())
# submission[['id', 'location']].to_csv('submission.csv', index=False)

## AMP 训练
1. AMP 是一种自动混合精度，称为 Automatic mixed precision, 目的是可以在神经网络推理过程中针对不同的层采用不同的数据精度计算， 可以节省显存加快速度。torch 中两种浮点型Tensor， float 16 和float 32。  
2. 为何需要AMP？ 因为HalfTensor 存储小，计算快 可以更好的利用CUDA设备的Tensor Core， 训练的时候可以很好的减小显存占用。因此训练的时候可以减少显存的占用（可以增加batchsize了），同时训练速度更快；
3. torch.HalfTensor的劣势就是：数值范围小（更容易Overflow / Underflow）、舍入误差（Rounding Error，导致一些微小的梯度信息达不到16bit精度的最低分辨率，从而丢失）


## Use BCELosswithlogit as loss function (binary cross entropy loss)
This class contains a sigmod+BCEloss:
$$Equation = \frac{1}{N}\sum_{n=1}^{N}l_n$$
where $l_n = -\omega[y_n * logx_n + (1-y_n) * log(1- x_n)]$, 其中w 是超参数

## 使用混合精度训练， 即 autocast + GradScaler

## 一些训练trick
1. gradient_accumulate_step: 如果显存不足，可以通过梯度累计来解决，举个例子， 假设原来的batch_size = 10, 数据总量为1000， 则需要100次training step，如果这样显存不够，则减小batch_size, 设置gradient_accumulate_step = 2, 则新更新的batch_size = 10/2 = 5, 则此时梯度更新的次数仍是100次， 则最终的training step = 200.
2. 指定对应的参数更新，比如weight decay不decay 哪一些层， 选定特定的层来更新
3. scheduler, 代表 根据batch 来进行学习率的调整

In [ ]:
def get_parameters_function(net, encoder_lr, decoder_lr, weight_decay):
    '''
    input_params: 
        1. net, the original network;
        2. encoder_lr, learning rate for encoder;
        3. decoder_lr, learning rate for decoder;
        4. weight_decay, decay rate
    Notice that here we write judge whole model as encoder and decoder, here Roberta is encoder, linear output is decoder.
    '''
    params = list(net.named_parameters())
    # point out value not need to decay
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_params = [
        # for encoder, here is Roberta info
        {"params": [p for n, p in net.model.named_parameters() if not any(nd in n for nd in no_decay)], 
        "lr": encoder_lr, "weight_decay":weight_decay},
        {"params": [p for n, p in net.model.named_parameters() if any(nd in n for nd in no_decay)], 
        "lr": encoder_lr, "weight_decay":0.0},
        # for decoder
        {"params":[p for n, p in net.named_parameters() if "model" not in n], 
        "lr": decoder_lr, "weight_decay":0.0}
    ]
    return optimizer_params

In [ ]:
from transformers import get_cosine_schedule_with_warmup
def get_scheduler_info(optimizer, train_step):
    '''
    This function is the to return the corresponding scheduler:
    input params:
        1. optimizer, which is the optimizer that need to be changed the learning rate
        2. train_step, which is called warm step, means don't change the lr after few epoches
    warm up 的作用： 对于刚开始训练的模型，由于weight是随机初始化的，若学习率过大，可能会导致震荡等因素的存在，所以在开始的时候，
    会让学习率从0不断上升，在这一段比较小的学习率下，模型可以慢慢趋于稳定，从而慢慢达到预设的初始学习率， 然后在慢慢下降初始学习率， 进行参数的学习
    '''
    # here use cos scheduler from hugging face
    scheduler = get_cosine_schedule_with_warmup(optimizer= optimizer, num_warmup_steps = 100, num_training_steps= train_step, num_cycles = 0.5)
    return scheduler

In [ ]:
def check_labels(labels):
    tmp = True
    for i in range(labels.shape[0]):
        #print(np.isin(labels[i,:].detach().numpy(),1).any())
        if not np.isin(labels[i,:].detach().numpy(),1).any():
            tmp = False
            return tmp
    return tmp

In [ ]:
# For this dataset
from torch.cuda import amp
def train_with_amp(net, train_set, criterion, optimizer, epochs,batch_size, scheduler, gradient_accumulate_step, max_grad_norm ,device):
    net.train()
    # instantiate a scalar object  
    print("train on " + str(device))
    ls = []
    enable_amp = True if "cuda" in device.type else False
    scaler = amp.GradScaler(enabled= enable_amp)
    net.to(device)
    global_step = 0
    train_iter = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
    for epoch in range(epochs):
        for idx, value in enumerate(train_iter):
            #sig = check_labels(value[2])
            input_ids = value[0].to(device)
            seg_ids = value[1].to(device)
            labels = value[2].to(device)
            #print(sig)
            # when forward process, use amp
            with amp.autocast(enabled= enable_amp):
                output = net(input_ids, seg_ids)
            loss = criterion(output.view(-1,1), labels.view(-1,1).float())
                # do the selection for non -1 data
            loss =  torch.masked_select(loss, labels.view(-1,1) != -1).mean()
            # prevent gradient to 0
            if gradient_accumulate_step > 1:
                # 如果显存不足，通过 gradient_accumulate 来解决
                loss = loss/gradient_accumulate_step
            
            # 放大梯度，避免其消失
            scaler.scale(loss).backward()
            # do the gradient clip
            gradient_norm = nn.utils.clip_grad_norm_(net.parameters(),max_grad_norm)
            if (idx + 1) % gradient_accumulate_step == 0:
                # 多少 step 更新一次梯度
                # 通过 scaler.step 来unscale 回梯度值， 如果气结果不是infs 和Nans， 调用optimizer.step()来更新权重
                # 否则忽略step调用， 保证权重不更新
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                global_step += 1
                scheduler.step()
            # 每100次计算 print 出一次loss
            if idx % 1000 == 0 or idx == len(train_iter) -1:
                with torch.no_grad():
                    print("==============Epochs "+ str(epoch) + " ======================")
                    print("loss: " + str(loss) + "; grad_norm: " + str(gradient_norm))
                    torch.save({"net":net.state_dict(), "epoches": epochs, "optimizer": optimizer.state_dict()}, "./model.params")
                    ls.append(loss.item())
            #print("successfully done one train")
    # Draw the loss
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(ls)

In [ ]:
def total_train(train_set, valid_Set):
    encoder_lr, decoder_lr = 2e-6, 2e-6
    weight_decay, eps = 0.01, 1e-6
    net = Roberta_baseline("roberta-large", 1, False,"../input/roberta-baseline-params")
    #dic = torch.load("../input/roberta-model-para/model.params")
   # net.load_state_dict(dic)
    num_class = 1
    num_epoches = 5
    batch_size = 6
#     name = "roberta-large"
#     net = Roberta_baseline(name, num_class)    
    optim_param = get_parameters_function(net, encoder_lr, decoder_lr, weight_decay)
    optimizer = torch.optim.AdamW(params= optim_param, lr = encoder_lr, eps=eps)
    # Since we will do some selection after loss function, just set reduction to None
    # And we process it by ourselves
    criterion = nn.BCEWithLogitsLoss(reduction="none")    
    scheduler = get_scheduler_info(optimizer, len(torch.utils.data.DataLoader(train_set, batch_size = batch_size)))
    train_with_amp(net, train_set, criterion, optimizer, num_epoches, batch_size, scheduler, 1,1500,torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    

In [ ]:
total_train(train_Set, None)